# HIRA 빅데이터 상담 챗봇 - 인터랙티브 인터페이스

**프로젝트**: HIRA 빅데이터 상담 챗봇  
**목적**: 학습된 모델을 직접 테스트하고 평가  
**환경**: JupyterLab

---

## 📋 목차

1. [모델 로드](#1-모델-로드)
2. [인터랙티브 질의응답](#2-인터랙티브-질의응답)
3. [배치 평가](#3-배치-평가)
4. [성능 분석](#4-성능-분석)
5. [웹 인터페이스 실행](#5-웹-인터페이스-실행)

## 1. 모델 로드

### 1.1 환경 설정

In [ ]:
# bitsandbytes 차단
import sys
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
sys.modules['bitsandbytes'] = None

import torch
from pathlib import Path
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

print("✅ 라이브러리 임포트 완료")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

### 1.2 경로 설정

In [ ]:
# 프로젝트 경로
PROJECT_ROOT = Path().absolute().parent

# 모델 경로
BASE_MODEL_PATH = PROJECT_ROOT / "solar_10.7b_package" / "model"
LORA_MODEL_PATH = PROJECT_ROOT / "workspace" / "models" / "solar_hira_lora" / "best_model"

# 또는 절대 경로 사용:
# BASE_MODEL_PATH = Path("/path/to/solar_10.7b_package/model")
# LORA_MODEL_PATH = Path("/path/to/workspace/models/solar_hira_lora/best_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("📂 경로 설정:")
print(f"  베이스 모델: {BASE_MODEL_PATH}")
print(f"  LoRA 모델: {LORA_MODEL_PATH}")
print(f"  Device: {device}")

# 경로 존재 확인
if not BASE_MODEL_PATH.exists():
    print(f"⚠️ 베이스 모델 경로가 존재하지 않습니다: {BASE_MODEL_PATH}")
if not LORA_MODEL_PATH.exists():
    print(f"⚠️ LoRA 모델 경로가 존재하지 않습니다: {LORA_MODEL_PATH}")
    print(f"  → 먼저 HIRA_Training_SOLAR_LoRA.ipynb를 실행하여 모델을 학습하세요.")

### 1.3 모델 로드

In [ ]:
print("🔄 모델 로드 중... (1-2분 소요)\n")

# 토크나이저 로드
print("[1/3] 토크나이저 로드...")
tokenizer = AutoTokenizer.from_pretrained(
    str(BASE_MODEL_PATH),
    trust_remote_code=True,
    local_files_only=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ 토크나이저 로드 완료")

# 베이스 모델 로드
print("\n[2/3] 베이스 모델 로드...")
base_model = AutoModelForCausalLM.from_pretrained(
    str(BASE_MODEL_PATH),
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True
)

print(f"✅ 베이스 모델 로드 완료 ({base_model.num_parameters() / 1e9:.2f}B params)")

# LoRA 어댑터 로드
print("\n[3/3] LoRA 어댑터 로드...")
model = PeftModel.from_pretrained(base_model, str(LORA_MODEL_PATH))
model.eval()

print("✅ LoRA 어댑터 로드 완료")

print("\n" + "="*80)
print("✅ 모델 준비 완료! 이제 질문을 시작할 수 있습니다.")
print("="*80)

### 1.4 추론 함수 정의

In [ ]:
def generate_response(
    question,
    instruction="건강보험 빅데이터 전문가로서 정확하게 답변하세요.",
    max_new_tokens=256,
    temperature=0.3,
    top_p=0.85,
    top_k=40,
    repetition_penalty=1.15
):
    """
    질문에 대한 답변 생성
    
    Parameters:
    -----------
    question : str
        사용자 질문
    instruction : str
        시스템 지시문
    max_new_tokens : int
        생성할 최대 토큰 수
    temperature : float
        샘플링 온도 (낮을수록 결정론적)
    top_p : float
        Nucleus sampling 임계값
    top_k : int
        Top-K sampling 크기
    repetition_penalty : float
        반복 페널티
    
    Returns:
    --------
    str : 생성된 답변
    """
    # 프롬프트 구성
    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{question}\n\n### Response:\n"
    
    # 토크나이징
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)
    
    # 생성
    start_time = datetime.now()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=3,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    generation_time = (datetime.now() - start_time).total_seconds()
    
    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Response 부분만 추출
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    
    # 메타데이터 추가
    metadata = {
        "generation_time": generation_time,
        "tokens_generated": len(outputs[0]) - len(inputs["input_ids"][0]),
        "temperature": temperature,
        "top_p": top_p
    }
    
    return response, metadata

print("✅ 추론 함수 정의 완료")

## 2. 인터랙티브 질의응답

### 2.1 간단한 테스트

In [ ]:
# 샘플 질문으로 테스트
test_question = "2023년 MRI 검사 건수는 얼마나 되나요?"

print(f"질문: {test_question}\n")
response, metadata = generate_response(test_question)
print(f"답변: {response}\n")
print(f"메타데이터:")
print(f"  - 생성 시간: {metadata['generation_time']:.2f}초")
print(f"  - 생성 토큰 수: {metadata['tokens_generated']}")

### 2.2 위젯 기반 인터랙티브 인터페이스

In [ ]:
# UI 위젯 생성
question_input = widgets.Textarea(
    value='',
    placeholder='질문을 입력하세요...',
    description='질문:',
    layout=widgets.Layout(width='100%', height='100px')
)

# 파라미터 조절 슬라이더
temperature_slider = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Temperature:',
    style={'description_width': 'initial'}
)

max_tokens_slider = widgets.IntSlider(
    value=256,
    min=64,
    max=512,
    step=32,
    description='Max Tokens:',
    style={'description_width': 'initial'}
)

top_p_slider = widgets.FloatSlider(
    value=0.85,
    min=0.5,
    max=1.0,
    step=0.05,
    description='Top-p:',
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='답변 생성',
    button_style='primary',
    icon='check'
)

output_area = widgets.Output()

# 버튼 클릭 핸들러
def on_generate_click(b):
    with output_area:
        clear_output()
        
        question = question_input.value.strip()
        
        if not question:
            print("⚠️ 질문을 입력해주세요.")
            return
        
        print(f"🔄 답변 생성 중...\n")
        
        # 답변 생성
        response, metadata = generate_response(
            question,
            max_new_tokens=max_tokens_slider.value,
            temperature=temperature_slider.value,
            top_p=top_p_slider.value
        )
        
        # 결과 출력 (HTML 스타일)
        display(HTML(f"""
        <div style="background-color: #f0f0f0; padding: 15px; border-radius: 10px; margin-bottom: 15px;">
            <h3 style="color: #2c3e50; margin-top: 0;">질문</h3>
            <p style="font-size: 16px; color: #34495e;">{question}</p>
        </div>
        
        <div style="background-color: #e8f5e9; padding: 15px; border-radius: 10px; margin-bottom: 15px;">
            <h3 style="color: #1b5e20; margin-top: 0;">답변</h3>
            <p style="font-size: 16px; color: #2e7d32; line-height: 1.6;">{response}</p>
        </div>
        
        <div style="background-color: #fff3e0; padding: 10px; border-radius: 5px;">
            <p style="margin: 5px 0; font-size: 14px;"><strong>생성 시간:</strong> {metadata['generation_time']:.2f}초</p>
            <p style="margin: 5px 0; font-size: 14px;"><strong>생성 토큰:</strong> {metadata['tokens_generated']}개</p>
            <p style="margin: 5px 0; font-size: 14px;"><strong>Temperature:</strong> {metadata['temperature']}</p>
        </div>
        """))

generate_button.on_click(on_generate_click)

# UI 레이아웃
ui = widgets.VBox([
    widgets.HTML("<h2>🏥 HIRA 빅데이터 상담 챗봇</h2>"),
    question_input,
    widgets.HTML("<h3>생성 파라미터</h3>"),
    temperature_slider,
    max_tokens_slider,
    top_p_slider,
    generate_button,
    output_area
])

display(ui)

### 2.3 명령줄 인터페이스

In [ ]:
# 간단한 대화형 인터페이스 (셀을 여러 번 실행 가능)
print("💬 질문을 입력하세요 (종료: 'quit')\n")

conversation_history = []

while True:
    question = input("\n질문: ").strip()
    
    if question.lower() in ['quit', 'exit', '종료', 'q']:
        print("\n대화를 종료합니다.")
        break
    
    if not question:
        continue
    
    response, metadata = generate_response(question)
    
    print(f"\n답변: {response}")
    print(f"\n[생성 시간: {metadata['generation_time']:.2f}초]")
    
    # 대화 기록 저장
    conversation_history.append({
        "question": question,
        "response": response,
        "timestamp": datetime.now().isoformat(),
        "metadata": metadata
    })

# 대화 기록 저장
if conversation_history:
    output_file = PROJECT_ROOT / "workspace" / "logs" / f"conversation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    output_file.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(conversation_history, f, ensure_ascii=False, indent=2)
    
    print(f"\n대화 기록 저장: {output_file}")

## 3. 배치 평가

### 3.1 미리 정의된 테스트 질문

In [ ]:
# 테스트 질문 세트
test_questions = [
    "2023년 MRI 검사 건수는 얼마나 되나요?",
    "DRG가 무엇인가요?",
    "고혈압 환자 수는 얼마나 되나요?",
    "건강보험 총 진료비는?",
    "요양병원과 종합병원의 차이는?",
    "본인부담금이란?",
    "신포괄수가제도를 설명해주세요",
    "건강검진 대상자는?",
    "암 환자 진료비 지원 제도는?",
    "처방전 유효기간은?"
]

print(f"테스트 질문 {len(test_questions)}개를 평가합니다...\n")

results = []

for i, question in enumerate(test_questions, 1):
    print(f"[{i}/{len(test_questions)}] {question}")
    
    response, metadata = generate_response(question, temperature=0.3)
    
    print(f"답변: {response[:100]}...")
    print(f"생성 시간: {metadata['generation_time']:.2f}초\n")
    
    results.append({
        "question": question,
        "response": response,
        "metadata": metadata
    })

print("\n✅ 배치 평가 완료")

# 결과 저장
result_file = PROJECT_ROOT / "workspace" / "logs" / f"batch_eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
result_file.parent.mkdir(parents=True, exist_ok=True)

with open(result_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"결과 저장: {result_file}")

### 3.2 결과 분석

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 생성 시간 분석
generation_times = [r['metadata']['generation_time'] for r in results]
tokens_generated = [r['metadata']['tokens_generated'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 생성 시간 분포
ax1.bar(range(len(generation_times)), generation_times, color='skyblue')
ax1.set_xlabel('질문 번호')
ax1.set_ylabel('생성 시간 (초)')
ax1.set_title('질문별 생성 시간')
ax1.axhline(np.mean(generation_times), color='red', linestyle='--', label=f'평균: {np.mean(generation_times):.2f}초')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 토큰 수 분포
ax2.bar(range(len(tokens_generated)), tokens_generated, color='lightgreen')
ax2.set_xlabel('질문 번호')
ax2.set_ylabel('생성 토큰 수')
ax2.set_title('질문별 생성 토큰 수')
ax2.axhline(np.mean(tokens_generated), color='red', linestyle='--', label=f'평균: {np.mean(tokens_generated):.1f}개')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 통계:")
print(f"  평균 생성 시간: {np.mean(generation_times):.2f}초")
print(f"  최소 생성 시간: {np.min(generation_times):.2f}초")
print(f"  최대 생성 시간: {np.max(generation_times):.2f}초")
print(f"  평균 토큰 수: {np.mean(tokens_generated):.1f}개")
print(f"  토큰/초: {np.mean(tokens_generated) / np.mean(generation_times):.1f}")

## 4. 성능 분석

### 4.1 답변 품질 평가 (수동)

In [ ]:
# 인간 평가를 위한 UI
def create_evaluation_ui(results):
    """
    각 답변에 대한 평가 UI 생성
    """
    evaluations = []
    
    for i, result in enumerate(results):
        print(f"\n{'='*80}")
        print(f"질문 {i+1}/{len(results)}")
        print(f"{'='*80}")
        print(f"\nQ: {result['question']}")
        print(f"\nA: {result['response']}")
        print("\n" + "-"*80)
        
        # 평가 입력
        print("\n평가 (1-5점):")
        print("  1: 매우 나쁨")
        print("  2: 나쁨")
        print("  3: 보통")
        print("  4: 좋음")
        print("  5: 매우 좋음")
        
        try:
            score = int(input("\n점수: ").strip())
            if score < 1 or score > 5:
                score = 3
        except:
            score = 3
        
        comment = input("코멘트 (선택): ").strip()
        
        evaluations.append({
            "question": result['question'],
            "response": result['response'],
            "score": score,
            "comment": comment
        })
        
        print("\n✅ 평가 저장됨")
    
    return evaluations

# 평가 실행 (선택사항)
# evaluations = create_evaluation_ui(results)

print("수동 평가 UI 준비 완료")
print("주석을 제거하여 평가를 시작하세요.")

### 4.2 파라미터 비교 실험

In [ ]:
# Temperature 비교
test_q = "DRG가 무엇인가요?"

temperatures = [0.1, 0.3, 0.5, 0.7, 1.0]

print(f"질문: {test_q}\n")
print("="*80)

for temp in temperatures:
    print(f"\n[Temperature = {temp}]")
    response, _ = generate_response(test_q, temperature=temp)
    print(f"{response}\n")
    print("-"*80)

## 5. 웹 인터페이스 실행

### 5.1 Flask 서버 시작 (백그라운드)

In [ ]:
# Flask 앱을 백그라운드에서 실행
# 주의: 주피터랩에서는 별도 터미널에서 실행하는 것을 권장

print("웹 인터페이스를 실행하려면:")
print("\n1. 터미널 열기")
print("2. 다음 명령 실행:")
print("\n   python 03_improved_interface.py")
print("\n3. 브라우저에서 접속:")
print("   http://localhost:8888")
print("\n또는 프록시 URL 사용 (Backend.AI 등):")
print("   http://<proxy-url>/proxy/8888/opnAI")

### 5.2 간단한 HTTP 서버 (Gradio)

In [ ]:
# Gradio 인터페이스 (선택사항)
# pip install gradio 필요

try:
    import gradio as gr
    
    def gradio_generate(question, temperature, max_tokens):
        """
        Gradio용 생성 함수
        """
        if not question.strip():
            return "질문을 입력해주세요."
        
        response, metadata = generate_response(
            question,
            temperature=temperature,
            max_new_tokens=max_tokens
        )
        
        result = f"{response}\n\n"
        result += f"[생성 시간: {metadata['generation_time']:.2f}초, "
        result += f"토큰: {metadata['tokens_generated']}개]"
        
        return result
    
    # Gradio UI
    iface = gr.Interface(
        fn=gradio_generate,
        inputs=[
            gr.Textbox(lines=3, placeholder="질문을 입력하세요...", label="질문"),
            gr.Slider(0.1, 1.0, value=0.3, step=0.1, label="Temperature"),
            gr.Slider(64, 512, value=256, step=32, label="Max Tokens")
        ],
        outputs=gr.Textbox(lines=10, label="답변"),
        title="🏥 HIRA 빅데이터 상담 챗봇",
        description="건강보험 관련 질문에 답변합니다.",
        examples=[
            ["2023년 MRI 검사 건수는?", 0.3, 256],
            ["DRG가 무엇인가요?", 0.3, 256],
            ["고혈압 환자 수는?", 0.3, 256]
        ]
    )
    
    # 서버 실행 (포트 7860)
    # iface.launch(share=False, server_port=7860)
    
    print("✅ Gradio 인터페이스 준비 완료")
    print("주석을 제거하고 iface.launch()를 실행하세요.")
    
except ImportError:
    print("⚠️ Gradio가 설치되지 않았습니다.")
    print("설치: pip install gradio")

## 완료!

이제 모델을 다양한 방법으로 테스트할 수 있습니다:

1. **위젯 인터페이스**: 위의 2.2 셀 사용
2. **명령줄 인터페이스**: 위의 2.3 셀 실행
3. **배치 평가**: 3.1 셀로 대량 테스트
4. **웹 인터페이스**: Flask 또는 Gradio

### 다음 단계

- 실제 HIRA 데이터로 재학습
- 더 많은 QA 쌍 추가
- 하이퍼파라미터 튜닝
- RAG 시스템 통합